In [ ]:
import math
from google.cloud import bigquery

In [ ]:
client = bigquery.Client.from_service_account_json('/content/drive/MyDrive/tech-cali-b2c-72b3e690e309-Compute-Engine.json')

project_id1 = 'tech-cali-b2c'
dataset_id1 = 'CE_ML_Layer'
table_id1 = 'Z_Scoring'

df_fi = client.query(f"SELECT * FROM `{project_id1}.{dataset_id1}.{table_id1}`").to_dataframe()

In [ ]:
project_id2 = 'tech-cali-b2c'
dataset_id2 = 'CE_ML_Layer'
table_id2 = 'Feature_Importance'

feature_imp = client.query(f"SELECT * FROM `{project_id2}.{dataset_id2}.{table_id2}`").to_dataframe()

In [ ]:
def map_score_bucket_label(score_bucket):
  if math.ceil(score_bucket) >= 8:
    return "8.Extremely High"
  elif math.ceil(score_bucket) >= 7:
    return "7.Very High"
  elif math.ceil(score_bucket) >= 6:
    return "6.High"
  elif math.ceil(score_bucket) >= 5:
    return "5.Above Medium"
  elif math.ceil(score_bucket) >= 4:
    return "4.Medium"
  elif math.ceil(score_bucket) >= 3:
    return "3.Below Medium"
  elif math.ceil(score_bucket) >= 2:
    return "2.Low"
  elif math.ceil(score_bucket) >= 1:
    return "1.Very Low"
  else:
    return "0.Not Applicable"

**Finding Metric Score**

In [ ]:
df_A_Score = feature_imp[['spotify_followers','Instagram_followers','Instagram_Genz_followers','TikTok_followers','TikTok_Genz_followers',
                          'Youtube_subscribers','twitter_followers','Youtube_Genz_subscribers']]
df_C_Score = feature_imp[['youtube_monthly_video_views','TikTok_total_views_for_top_n_tracks','spotify_monthly_listeners','twitter_retweets']]
df_E_Score = feature_imp[['TikTok_avg_engagements_per_post','Instagram_avg_engagements_per_post','facebook_likes','facebook_talks',
                          'Youtube_avg_engagements_per_post']]

In [ ]:
new1 = df_A_Score.transpose()
new2 = df_C_Score.transpose()
new3 = df_E_Score.transpose()

In [ ]:
new_df_A = new1.sort_values(0, ascending = True)
new_df_C = new2.sort_values(0, ascending = True)
new_df_E = new3.sort_values(0, ascending = True)

In [ ]:
new_df_A["Metric_Score"] = [1,2,3,4,5,6,7,8]
new_df_C["Metric_Score"] = [1,2,3,4]
new_df_E["Metric_Score"] = [1,2,3,4,5]

**Finding New Weights**

In [ ]:
new_df_A["New_Weights_%"] = new_df_A['Metric_Score']/sum(new_df_A['Metric_Score'])
new_df_C["New_Weights_%"] = new_df_C['Metric_Score']/sum(new_df_C['Metric_Score'])
new_df_E["New_Weights_%"] = new_df_E['Metric_Score']/sum(new_df_E['Metric_Score'])

**Finding New Weighted Average Score**

In [ ]:
df_fi['New_Weighted_Average_Score_twitter_followers'] = df_fi['twitter_followers_zscore'] * new_df_A['New_Weights_%']['twitter_followers']
df_fi['New_Weighted_Average_Score_spotify_followers'] = df_fi['spotify_followers_zscore'] * new_df_A['New_Weights_%']['spotify_followers']
df_fi['New_Weighted_Average_Score_Instagram_followers'] = df_fi['Instagram_followers_zscore'] * new_df_A['New_Weights_%']['Instagram_followers']
df_fi['New_Weighted_Average_Score_TikTok_Genz_followers'] = df_fi['TikTok_Genz_followers_zscore'] * new_df_A['New_Weights_%']['TikTok_Genz_followers']
df_fi['New_Weighted_Average_Score_Instagram_Genz_followers'] = df_fi['Instagram_Genz_followers_zscore'] * new_df_A['New_Weights_%']['Instagram_Genz_followers']
df_fi['New_Weighted_Average_Score_Youtube_Genz_subscribers'] = df_fi['Youtube_Genz_subscribers_zscore'] * new_df_A['New_Weights_%']['Youtube_Genz_subscribers']
df_fi['New_Weighted_Average_Score_Youtube_subscribers'] = df_fi['Youtube_subscribers_zscore'] * new_df_A['New_Weights_%']['Youtube_subscribers']
df_fi['New_Weighted_Average_Score_TikTok_followers'] = df_fi['TikTok_followers_zscore'] * new_df_A['New_Weights_%']['TikTok_followers']

#----------------------------------------------------------------------------------------------------------------------------------------#


df_fi['New_Weighted_Average_Score_twitter_retweets'] = df_fi['twitter_retweets_zscore'] * new_df_C['New_Weights_%']['twitter_retweets']
df_fi['New_Weighted_Average_Score_youtube_monthly_video_views'] = df_fi['youtube_monthly_video_views_zscore'] * new_df_C['New_Weights_%']['youtube_monthly_video_views']
df_fi['New_Weighted_Average_Score_spotify_monthly_listeners'] = df_fi['spotify_monthly_listeners_zscore'] * new_df_C['New_Weights_%']['spotify_monthly_listeners']
df_fi['New_Weighted_Average_Score_TikTok_total_views_for_top_n_tracks'] = df_fi['TikTok_total_views_for_top_n_tracks_zscore'] * new_df_C['New_Weights_%']['TikTok_total_views_for_top_n_tracks']

#---------------------------------------------------------------------------------------------------------------------------------------#

df_fi['New_Weighted_Average_Score_Instagram_avg_engagements_per_post'] = df_fi['Instagram_avg_engagements_per_post_zscore'] * new_df_E['New_Weights_%']['Instagram_avg_engagements_per_post']
df_fi['New_Weighted_Average_Score_Youtube_avg_engagements_per_post'] = df_fi['Youtube_avg_engagements_per_post_zscore'] * new_df_E['New_Weights_%']['Youtube_avg_engagements_per_post']
df_fi['New_Weighted_Average_Score_TikTok_avg_engagements_per_post'] = df_fi['TikTok_avg_engagements_per_post_zscore'] * new_df_E['New_Weights_%']['TikTok_avg_engagements_per_post']
df_fi['New_Weighted_Average_Score_facebook_talks'] = df_fi['facebook_talks_zscore'] * new_df_E['New_Weights_%']['facebook_talks']
df_fi['New_Weighted_Average_Score_facebook_likes'] = df_fi['facebook_likes_zscore'] * new_df_E['New_Weights_%']['facebook_likes']

#----------------------------------------------------------------------------------------------------------------------------------------#

**Finding the Audience Score, Content Performance Score, Engagement Score, and Final Score**

In [ ]:
df_fi['Audience_Score'] = df_fi['New_Weighted_Average_Score_twitter_followers'] + df_fi['New_Weighted_Average_Score_spotify_followers'] + df_fi['New_Weighted_Average_Score_Instagram_followers'] +  \
                          df_fi['New_Weighted_Average_Score_TikTok_Genz_followers'] + df_fi['New_Weighted_Average_Score_Instagram_Genz_followers'] +  \
                          df_fi['New_Weighted_Average_Score_Youtube_Genz_subscribers'] + df_fi['New_Weighted_Average_Score_Youtube_subscribers'] + df_fi['New_Weighted_Average_Score_TikTok_followers']
#----------------------------------------------------------------------------------------------------------------------------------------#


df_fi['Content_Performance_Score'] = df_fi['New_Weighted_Average_Score_twitter_retweets'] + df_fi['New_Weighted_Average_Score_youtube_monthly_video_views'] +  \
                                      df_fi['New_Weighted_Average_Score_spotify_monthly_listeners'] + df_fi['New_Weighted_Average_Score_TikTok_total_views_for_top_n_tracks']
#---------------------------------------------------------------------------------------------------------------------------------------#

df_fi['Engagement_Score'] = df_fi['New_Weighted_Average_Score_Instagram_avg_engagements_per_post'] + df_fi['New_Weighted_Average_Score_Youtube_avg_engagements_per_post'] +  \
                            df_fi['New_Weighted_Average_Score_TikTok_avg_engagements_per_post'] + df_fi['New_Weighted_Average_Score_facebook_talks'] +  \
                            df_fi['New_Weighted_Average_Score_facebook_likes']
#----------------------------------------------------------------------------------------------------------------------------------------#


In [ ]:
df_fi['Final_Selection_Score'] = (df_fi['Audience_Score'] + df_fi['Content_Performance_Score'] + df_fi['Engagement_Score'])*(1/3)

In [ ]:
for col in list(df_fi.columns):
  if col.endswith('_Score'):
    col_score_bucket_label = col + '_Label'
    df_fi[col_score_bucket_label] = df_fi[col].apply(lambda x: map_score_bucket_label(x))

In [ ]:
from google.oauth2 import service_account
import pandas_gbq

# Define your BigQuery project ID and credentials
project_id = 'tech-cali-b2c'
credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/tech-cali-b2c-72b3e690e309-Compute-Engine.json')

# Define the BigQuery table name and dataset ID
dataset_id = 'CE_ML_Layer'
table_name = 'New_Weights_Score'

# Write the dataframe to BigQuery
pandas_gbq.to_gbq(df_fi, f'{dataset_id}.{table_name}', project_id=project_id, if_exists='replace', credentials=credentials)

print('Data moved to BigQuery successfully!')

100%|██████████| 1/1 [00:00<00:00, 5607.36it/s]

Data moved to BigQuery successfully!
